In [52]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [54]:
from typing import Tuple

import pandas as pd
import numpy as np
import isodate

from plotting import plot_ts
from load_dataset import Dataset
from model import TimeSeriesPredictor

### Какие ряды будем тестировать?

* длинный ряд с сезонностью  
* короткий ряд с сезонностью  
* короткий ряд с сезонностью и трендом  
* случайное блуждание  
* средне зашумленный ряд
* "шумный" ряд  

In [55]:
ds = Dataset('../data/dataset/')

In [56]:
long = ds['daily-min-temperatures.csv']

/Users/technogleb/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [57]:
plot_ts(long)

Loading BokehJS ...

In [58]:
short_season = ds['hour_3019.csv'][300:]

/Users/technogleb/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [59]:
plot_ts(short_season)

Loading BokehJS ...

In [60]:
short_season_trend = ds['international-airline-passengers.csv']

/Users/technogleb/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [61]:
plot_ts(short_season_trend)

Loading BokehJS ...

In [62]:
random_walk = ds['dow_jones_0.csv']

/Users/technogleb/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [63]:
plot_ts(random_walk)

Loading BokehJS ...

In [64]:
medium_noize = ds['hour_3426.csv'][300:]

/Users/technogleb/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [65]:
plot_ts(medium_noize)

Loading BokehJS ...

In [66]:
full_noize = ds['day_1574.csv']

/Users/technogleb/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [67]:
plot_ts(full_noize)

Loading BokehJS ...

### Какие модели будем тестировать?

* скользящее среднее
* экспоненциальное сглаживание
* autoArima
* линейная регрессия
* линейная регрессия с L1 регуляризацией (Ridge)
* RandomForeset
* градиентный бустинг
* полносвязная нейросеть с одним лагом в качестве горизонта прогнозирования
* полносвязная нейросеть с произвольным количеством лагов в качестве горизонта прогнозирования

In [68]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### По каким метрикам будем сравнивать?

* mse
* mae
* R2
* mape - если не будет ломаться на нулях
* mase

In [69]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score

from metrics import mase

### По какой методике будем тестировать?

* 70% трейн, 30% тест
* Out-of-sample, чтобы посмотреть как модель предсказывает "вдолгую"

`ВНИМАНИЕ`
Чтобы сделать корректный прогноз на тестовую выборку в режиме out-of-sample с учетом возможных пропусков, вам необходимо сделать прогноз от первого до последнего timestamp-a включительно, после чего взять нужные timestamp из теста (реализуй это в функции задания 1б).  
Нельзя просто написать `predictor.forecast(len(test))`.


* In-Sample, чтобы посмотреть как модель предсказывает на одну точку вперед
* Для поиска гиперпараметров можно делать кроссвалидацию на тесте по метрике mse

### Задание 1(а). Напишите функцию, разбивающую на train и test

In [70]:
def train_test_split(ts: pd.Series, ratio: float = 0.7) -> Tuple[pd.Series]:
    # ваш код здесь
    split_idx = int(len(ts) * ratio)
    ts_train, ts_test = ts[:split_idx], ts[split_idx:]
    return ts_train, ts_test

### Задание 1(б). Напишите функцию для получения численного горизонта прогнозирования через первый и последний timestamp-ы выборки.

В данной функции горизонт прогнозирования задается первым и последним timestamp-ом, а также гранулярностью ряда. Нам нужно получить горизонт прогнозирования в виде целового числа лагов, на которые нужно сделать прогноз.

In [71]:
def calculate_h(start: pd.Timestamp, end: pd.Timestamp, granularity: str) -> int:
    # ваш код здесь
    h = (end - start) / isodate.parse_duration(granularity) + 1
    return int(h)

### Зададим соответствие гранулярностей для наших рядов.

In [72]:
granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'
}

In [73]:
import math

from pandas._libs.tslibs.timestamps import Timestamp


def get_month_sin(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.sin(theta)


def get_month_cos(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.cos(theta)


def get_day_sin(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.sin(theta)


def get_day_cos(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.cos(theta)


def get_dayofweek_sin(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.sin(theta)


def get_dayofweek_cos(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.cos(theta)


def get_hour_sin(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.sin(theta)


def get_hour_cos(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.cos(theta)


def get_minute_sin(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.sin(theta)


def get_minute_cos(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.cos(theta)


datetime_mappers = {
    'month_sin': get_month_sin,
    'month_cos': get_month_cos,
    'day_sin': get_day_sin,
    'day_cos': get_day_cos,
    'dayofweek_sin': get_dayofweek_sin,
    'dayofweek_cos': get_dayofweek_cos,
    'hour_sin': get_hour_sin,
    'hour_cos': get_hour_cos,
    'minute_sin': get_minute_sin,
    'minute_cos': get_minute_cos,
}

### Задание 2. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через TimeSeriesPredictor.

* принмает на вход исходный ряд, гранулярность, количество лагов, модель, а также **kwargs, в которые мы будем передавать параметры модели

* разбивает ряд на train/test

* создает инстанс TimeSeriesPredictor с нужными параметрами

* обучает предиктор на трейне

* делает out_of_sample и in_sample прогноз

* возвращает train, test, in_sample, out_of_sample

In [74]:
def make_pipeline(
    ts: pd.Series,
    granularity: str,
    model: callable,
    num_lags=24,
    use_mappers=True,
    **kwargs
) -> Tuple[pd.Series]:
    
    # your code here
    
    return train, test, in_sample, out_of_sample

### Задание 3. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через auto_arima

* функция должна принимать исходный временной ряд, период сезонности, параметры дифференцирования d, D и boolean параметр seasonal, данные параметры будут являться для нас гиперпараметрами, все остальное за нас должна найти auto_arima

* разбивает на train, test

* обучает arima на train при помощи вызова функции auto_arima из библиотеки pmdarima с переданными параметрами и со следующими зафиксированными параметрами: `max_p=3, max_q=3, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True`

* в качестве out_of_sample прогноза просто вызовите метод predict

* в качестве in_sample прогноза обучите модель заново на всём ряде методом `fit`, вызовите метод predict_in_sample и в качестве прогноза возьмите `in_sample_predictions(-len(test):)`

* возвращает train, test, in_sample, out_of_sample (не забудьте сделать их pd.Series с нужным индексом!!)

In [75]:
def make_pipeline_arima(ts: pd.Series, period: int, d: int = 1, D: int = 1, seasonal: bool = True) -> Tuple[pd.Series]:
    # your code here
    
    return train, test, in_sample, out_of_sample

### Задание 4. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через полносвязную сеть.

* функция должна принимать исходный временной ряд, количество входных лагов для формирования признаков, количество выходных лагов для формирования таргетов, количество скрытых слоев и количество нейронов на каждом слое
* подготавилвает выборку согласно переданным параметрам num_lags_in, num_lags_out, используя ранее написанную нами функцию transform_ts_into_matrix (приведена ниже)
* разбивает данные на трейн и тест
* создает модель Sequential с нужной архитектурой
    - num_lags_in задает значение параметра input_dims на первом слое
    - num_lags_out задает количество нейронов на последнем слое
    - количество нейронов на всех слоях от первого до предпоследнего задается в кортеже units
    - epochs задает количество епох для обучения
* обучает модель на трейне
* делает in_sample прогноз на тесте вызовом метода predict (обратите внимание, что вызов предикта должен осуществляться с шагом num_lags_out)
* делает out_of_sample прогноз рекурсивно c шагом num_lags_out, добавляя спрогнозированные точки в новые объекты (см. аналогично TimeSeriesPredictor)

In [76]:
def transform_ts_into_matrix(ts: pd.Series, num_lags_in: int, num_lags_out: int) -> Tuple[np.array, np.array]:
    """
    Данная функция должна пройтись скользящим окном по временному ряду и для каждых
    num_lags_in точек в качестве признаков собрать num_lags_out следующих точек в качестве таргета.
    
    Вернуть два np.array массива из X_train и y_train соответственно
    """
    sequence = ts.values
    X, y = list(), list()
    i = 0
    outer_idx = num_lags_out
    while outer_idx < len(sequence):
        inner_idx = i + num_lags_in
        outer_idx = inner_idx + num_lags_out
        X_, y_ = sequence[i:inner_idx], sequence[inner_idx:outer_idx]
        X.append(X_)
        y.append(y_)
        i += 1
    return np.array(X), np.array(y)

In [77]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense


def make_pipeline_fullyconnected(
    ts: pd.Series,
    num_lags_in: int,
    num_lags_out: int,
    hidden_layers: int,
    units: tuple[int],
    epochs: int
) -> Tuple[pd.Series]:
    # your code here
    return train, test, in_sample, out_of_sample

### Задание 5. Напишите функцию, имплементирующую поиск гиперпараметров по сетке. 

* функция должна принимать на вход ряд, гранулярность, модель, дефолтное количество лагов, сетку параметров (словарь)
* после написанного мной кода, функция должна с текущими параметрами запустить пайплайн (функция make_pipeline), получив таким образом прогнозы in_sample и out_of_sample
* посчитать mse для in_sample и out_of_sample прогноза, запомнить их в соответствующие словари
* вернуть лучшие параметры для in_sample и out_of_sample прогнозов

Замечания
* не забудьте, что в сетку параметров можно передавать также num_lags
* если в ряде ts_test есть пропуски, индекс прогноза out_of_sample будет не совпадать c индексом реальных данных, в таком случае, замените индекс out_of_sample прогноза индексом ts_test

In [78]:
from itertools import product

def hyperparameters_search(ts, granularity, model, num_lags, param_grid, verbose=False, use_mappers=True):
    
    statistics_in_sample, statistics_out_of_sample = {}, {}
    
    for param_tuple in product(*param_grid.values()):
        pass
        # your code here
    
    return best_in_sample, best_out_of_sample, [r2_score(test, in_sample), r2_score(test, out_of_sample)]

### Задание 6. "Прогоните" все алгоритмы на всех рядах и получите сводную таблицу результатов по всем метрикам, постройте также графики прогнозов. 

In [79]:
# your code here